In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)

In [2]:
def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2),
                torch.randn(2, 1, self.hidden_dim // 2))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        # return score, tag_seq
        return tag_seq

In [5]:
# data
import numpy as np
import pandas as pd

PARTICIPANT_ID = 1 #4348

df_total_csv = pd.read_csv('/Users/apple/Desktop/deep_eye/dataset/input_right.csv',usecols=['participant_ID','Q_ID','text','fixation1'])
#df_csv=df_csv[(df_csv['participant_ID']==PARTICIPANT_ID)] # take 1st participant data
#df_csv=df_csv.drop(['participant_ID'],axis=1)

In [36]:
total_p_list = []
for p in range(60):
    df_csv = df_total_csv[(df_total_csv['participant_ID'] == p+1)] # take 1st participant data
    total_q_list = []
    q_index_list = sorted(list(set(df_csv['Q_ID'])))
    
    for q in q_index_list:
        df_q = df_csv[df_csv['Q_ID'] == q]
        text_list = list(df_q['text'])
        label_list = list(df_q['fixation1'])
        q_list = [text_list, label_list]
        total_q_list.append(q_list)
    #total_p_list.append(total_q_list)
    total_p_list = total_p_list + total_q_list
print(
    '\n',total_q_list[0], # first data with text and label
    '\n',total_q_list[0][0], # first data's text
    '\n',total_q_list[0][1], # first data's label
)



 [['兩', '個', '量筒', '各', '裝', '50', 'cc', '水', '與', '50', 'cc', '酒精', '混合', '後', '為', '多少', 'cc', '等於', '100', 'cc', '少於', '100', 'cc', '接近', '100', 'cc'], [0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0]] 
 ['兩', '個', '量筒', '各', '裝', '50', 'cc', '水', '與', '50', 'cc', '酒精', '混合', '後', '為', '多少', 'cc', '等於', '100', 'cc', '少於', '100', 'cc', '接近', '100', 'cc'] 
 [0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0]


In [37]:
total_p_list

[[['沙罐',
   '置入',
   '不同',
   '濃度',
   '鹽水',
   '中',
   '所',
   '受',
   '浮力',
   '會',
   '如何',
   '相同',
   '濃度',
   '愈',
   '高',
   '所',
   '受',
   '浮力',
   '愈',
   '大',
   '濃度',
   '愈',
   '高',
   '所',
   '受',
   '浮力',
   '愈',
   '小'],
  [0,
   0,
   1,
   1,
   1,
   0,
   0,
   0,
   1,
   0,
   1,
   0,
   1,
   1,
   1,
   0,
   1,
   1,
   1,
   1,
   1,
   1,
   0,
   0,
   0,
   1,
   1,
   1]],
 [['外加',
   '電池',
   '將',
   '碳棒',
   '及',
   '銅棒',
   '放入',
   '硫酸銅',
   '𥚃',
   '將',
   '發生',
   '什麼',
   '反應',
   '電解',
   '反應',
   '電池',
   '反應',
   '氧化還原',
   '反應'],
  [0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1]],
 [['以',
   '針筒',
   '吸取',
   '熱水',
   '並',
   '封住',
   '針頭',
   '將',
   '針頭',
   '活塞',
   '向',
   '外',
   '拉出',
   '裡面',
   '的',
   '水',
   '會',
   '有',
   '何',
   '現象',
   '水位',
   '不變',
   '水',
   '沸騰',
   '水',
   '向',
   '上',
   '四散',
   '各',
   '處'],
  [0,
   0,
   0,
   0,
   0,
   1,
   1,
   0,
   1,
   0,
   0,
   0,
   1,
   1,
   0,
   1,
   1,
   

In [39]:
START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 4 #5
HIDDEN_DIM = 4 #4


training_data = total_p_list#[:37]
#testing_data = total_q_list[37:]

# word_to_ix # unique word and its representing number
word_to_ix = {}
for sentence, tags in training_data:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

tag_to_ix = {0: 0, 1: 1, START_TAG: 2, STOP_TAG: 3}

model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

# Check predictions before training
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
    precheck_tags = torch.tensor([tag_to_ix[t] for t in training_data[0][1]], dtype=torch.long)
    print('real label：'+ str(precheck_tags.tolist()))
    print('untrained label prediction：'+ str(model(precheck_sent)))

print('=============start BiLSTM+CRF model training=============')
# Make sure prepare_sequence from earlier in the LSTM section is loaded
for epoch in range(5): # epoch number
    for sentence, tags in training_data:
        # Pytorch accumulates gradients, we need to clear them out before each instance
        model.zero_grad()

        # turn inputs to Tensors of word indices
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long)

        # run forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # compute loss, gradients, and update the parameters by calling optimizer.step()
        loss.backward()
        optimizer.step()
    if epoch%50 == 0:
        print(f'the{epoch}th epoch Loss:{loss[0]}')


output_path = 'ner_trained_model.cpt'
torch.save(model, output_path)
print('=============trained model saved=============\n\n')



print('=============load trained model and predict=============')
model_path = 'ner_trained_model.cpt'
trained_ner_model = torch.load(model_path)
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
    print('trained label prediction：' + str(model(precheck_sent)))


real label：[0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1]
untrained label prediction：[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]
=============start BiLSTM+CRF model training=============
the0th epoch Loss:3.427607536315918
=============trained model saved=============


=============load trained model and predict=============
trained label prediction：[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [10]:
# # accuracy from scratch
# total_wrong = []
# acccuracy_score = []
# for q in range(47):
#     precheck_sent = prepare_sequence(training_data[q][0], word_to_ix)
#     predict = model(precheck_sent)
#     answer = training_data[q][1]
#     accuracy = sum([1 for i,j in zip(predict,answer) if i==j])/len(answer)
#     print(str(q+1) + " accuracy:", round(accuracy,2))
#     acccuracy_score.append(accuracy)
#     wrong_list = [i for i,j in zip(predict,answer) if i!=j] # wrong answer
#     total_wrong.append(wrong_list)

In [40]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# training predict score
total_result_list = []
for q in range(len(training_data)):
    precheck_sent = prepare_sequence(training_data[q][0], word_to_ix)
    predict = model(precheck_sent)
    answer = training_data[q][1]
    # print(training_data[q][0])
    # print(predict)
    # print(answer)
    # print("===============================")
    accuracy = accuracy_score(answer, predict)
    precision = precision_score(answer, predict)
    recall = recall_score(answer, predict)
    f1score = f1_score(answer, predict)
    result_list = [accuracy, precision, recall, f1score]
    result_list = [round(elem, 2) for elem in result_list]
    total_result_list.append(result_list)

['沙罐', '置入', '不同', '濃度', '鹽水', '中', '所', '受', '浮力', '會', '如何', '相同', '濃度', '愈', '高', '所', '受', '浮力', '愈', '大', '濃度', '愈', '高', '所', '受', '浮力', '愈', '小']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1]
['外加', '電池', '將', '碳棒', '及', '銅棒', '放入', '硫酸銅', '𥚃', '將', '發生', '什麼', '反應', '電解', '反應', '電池', '反應', '氧化還原', '反應']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1]
['以', '針筒', '吸取', '熱水', '並', '封住', '針頭', '將', '針頭', '活塞', '向', '外', '拉出', '裡面', '的', '水', '會', '有', '何', '現象', '水位', '不變', '水', '沸騰', '水', '向', '上', '四散', '各', '處']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0]
['雌蕊', '的', '花柱', '內', '有', '多少', '個', '花粉管', '一', '個', '兩', '個', '很多', '個']
[0, 0, 0, 0, 0, 0, 0, 0

/Users/apple/Documents/deepeye_env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/apple/Documents/deepeye_env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/apple/Documents/deepeye_env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/apple/Documents/deepeye_env/lib/python3.9/site-pack

['將', '罐子', '綁上', '乒乓球', '當', '往', '右', '移動', '罐子', '罐', '內', '乒乓球', '會', '有', '何', '現象', '向', '右', '跑', '向', '左', '跑', '停', '在', '原處', '不', '動']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0]
['在', '兩', '個', '懸掛', '的', '氣球', '中間', '用', '吹風機', '吹氣', '請問', '氣球', '會', '有', '什麼', '變化', '兩', '球', '分開', '兩', '球', '靠近', '兩', '球', '位置', '不變']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0]
['在', '鋅銅電池', '反應', '中', '檢流計', '指針', '的', '指向', '電子流向', '是', '向', '左', '不', '流動', '向', '右']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1]
['在', '封閉', '針筒', '中', '置入', '一', '小', '氣球', '將', '活塞', '向', '外', '拉', '後', '氣球', '會', '有', '何', '變化', '氣球', '縮小', '氣球', '脹大', '氣球', '大小', '不變']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

/Users/apple/Documents/deepeye_env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/apple/Documents/deepeye_env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/apple/Documents/deepeye_env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/apple/Documents/deepeye_env/lib/python3.9/site-pack

['砝碼', '置於', '一', '個', '和', '串聯', '二', '個', '相同', '的', '彈簧', '下', '不計', '彈簧', '重量', 'A', 'B', 'C', '彈簧', '伸長量', '一', '個', '彈簧', 'C', '比較', '長', '串聯', '之', '彈簧', 'A', '和', 'B', '比較', '長', 'A', '=', 'B', '=', 'C']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1]
['將', '金屬', '銅', '放', '在', '硫酸銅', '溶液', '中', '會', '發生', '什麼', '反應', '產生', '電流', '不', '產生', '電流', '產生', '新', '物質']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0]
['水', '沸騰', '後', '將', '火', '移開', '封住', '瓶口', '澆', '冷水', '瓶', '內', '水', '會', '如何', '沸騰', '凝結', '不會', '改變']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1]
['攀藤植物', '沒有', '支撐物', '時', '會', '如何', '生長', '會', '依', '在', '地面上', '長', '會', '直直', '向', '上', '長

...

/Users/apple/Documents/deepeye_env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/apple/Documents/deepeye_env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/apple/Documents/deepeye_env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/apple/Documents/deepeye_env/lib/python3.9/site-pack

['在', '兩', '個', '懸掛', '的', '氣球', '中間', '用', '吹風機', '吹氣', '請問', '氣球', '會', '有', '什麼', '變化', '兩', '球', '分開', '兩', '球', '靠近', '兩', '球', '位置', '不變']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['用', '廣口瓶', '蓋上', '點燃', '的', '蠟燭', '熄滅', '後', '水位', '上升', '高度', '為何', '5', '>', '3', '>', '1', '1', '=', '3', '=', '5', '1', '>', '3', '>', '5']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['以', '綠光', '與', '藍光', '照射', '水蘊草', '哪', '種', '色光', '會', '使', '水蘊草', '產生', '較少', '氣泡', '綠光', '藍光', '兩', '者', '差不多']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0]
['法碼', '置於', '一', '個', '和', '並聯', '二', '個', '相同', '彈簧', '下', '不計', '彈簧', '重量', 'A', 'B', 'C', '彈簧', '伸長量', '一', '個', '彈簧', 'A', '比較', '長', '並聯', '之',

/Users/apple/Documents/deepeye_env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/apple/Documents/deepeye_env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/apple/Documents/deepeye_env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/apple/Documents/deepeye_env/lib/python3.9/site-pack

['將', '幼苗', '平放', '幼苗', '上側', '或是', '下側', '生長素', '濃度', '較', '高', '使', '根', '向', '下', '長', '上側', '下側', '一樣', '多']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0]
['在', '寶特瓶', '側邊', '鑽洞', '如', '下', '不蓋', '瓶蓋', '水', '會', '從', '洞', '口', '溢出', '蓋上', '瓶蓋', '後', '會', '有', '何', '變化', '水', '繼續', '漏出', '水', '不', '漏出', '與', '不蓋', '瓶蓋', '時', '相同']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0]
['將', '金屬', '銅', '放', '在', '硫酸銅', '溶液', '中', '會', '發生', '什麼', '反應', '產生', '電流', '不', '產生', '電流', '產生', '新', '物質']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0]
['將', '鋅', '與', '銅', '放', '在', '酒精', 'C2H5OH', '中', '會', '發生', '什麼', '反應', '產生', '電流', '不', '產生', '電流', '產生', '新', '物質']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

/Users/apple/Documents/deepeye_env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/apple/Documents/deepeye_env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/apple/Documents/deepeye_env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/apple/Documents/deepeye_env/lib/python3.9/site-packages/skl

In [41]:
total_result_df = pd.DataFrame(total_result_list)
total_result_df.columns = ['accuracy', 'precision', 'recall', 'f1score']
total_result_df

,accuracy,precision,recall,f1score
0,0.39,0.0,0.0,0.0
1,0.37,0.0,0.0,0.0
2,0.57,0.0,0.0,0.0
3,0.64,0.0,0.0,0.0
4,0.59,0.0,0.0,0.0
...,...,...,...,...
1352,0.61,0.0,0.0,0.0
1353,0.73,0.0,0.0,0.0
1354,0.85,0.0,0.0,0.0
1355,0.76,0.0,0.0,0.0


In [42]:
total_result_df.mean(axis=0)

accuracy     0.682329
precision    0.000000
recall       0.000000
f1score      0.000000
dtype: float64

---
testing data show

In [34]:
# # for testing data
# word_to_ix = {}
# for sentence, tags in testing_data:
#     for word in sentence:
#         if word not in word_to_ix:
#             word_to_ix[word] = len(word_to_ix)

In [35]:
total_result_list = []
for q in range(len(testing_data)):
    precheck_sent = prepare_sequence(testing_data[q][0], word_to_ix)
    predict = model(precheck_sent)
    answer = testing_data[q][1]
    accuracy = accuracy_score(answer, predict)
    precision = precision_score(answer, predict)
    recall = recall_score(answer, predict)
    f1score = f1_score(answer, predict)
    result_list = [accuracy, precision, recall, f1score]
    result_list = [round(elem, 2) for elem in result_list]
    total_result_list.append(result_list)

In [36]:
total_result_df = pd.DataFrame(total_result_list)
total_result_df.columns = ['accuracy', 'precision', 'recall', 'f1score']
total_result_df

,accuracy,precision,recall,f1score
0,0.37,0.22,0.29,0.25
1,0.44,0.60,0.41,0.49
2,0.38,0.45,0.26,0.33
3,0.52,0.33,0.40,0.36
4,0.58,0.54,0.64,0.58
5,0.78,0.60,1.00,0.75
6,0.50,0.40,0.44,0.42
7,0.63,0.45,0.56,0.50
8,0.52,0.64,0.50,0.56
9,0.67,0.88,0.58,0.70


In [26]:
total_result_df.mean(axis=0)

accuracy     0.953333
precision    0.963889
recall       0.911667
f1score      0.935000
dtype: float64

In [28]:
total_result_list = []
for q in range(len(testing_data)):
    precheck_sent = prepare_sequence(testing_data[q][0], word_to_ix)
    predict = model(precheck_sent)
    answer = testing_data[q][1]
    print(testing_data[q][0])
    print(predict)
    print(answer)
    print("===============================")
    # accuracy = accuracy_score(answer, predict)
    # precision = precision_score(answer, predict)
    # recall = recall_score(answer, predict)
    # f1score = f1_score(answer, predict)
    # result_list = [accuracy, precision, recall, f1score]
    # result_list = [round(elem, 2) for elem in result_list]
    # total_result_list.append(result_list)

In [18]:
# # draw confusion matrix
# import matplotlib.pyplot as plt
# conf_matrix = confusion_matrix(y_true=answer, y_pred=predict)
# fig, ax = plt.subplots(figsize=(5, 5))
# ax.matshow(conf_matrix, cmap=plt.cm.Oranges, alpha=0.3)
# for i in range(conf_matrix.shape[0]):
#     for j in range(conf_matrix.shape[1]):
#         ax.text(x=j, y=i,s=conf_matrix[i, j], va='center', ha='center', size='xx-large')

# plt.xlabel('Predictions', fontsize=18)
# plt.ylabel('Actuals', fontsize=18)
# plt.title('Confusion Matrix', fontsize=18)
# plt.show()